In [ ]:
!git clone https://github.com/mcdermottLab/pycochleagram.git

In [ ]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg -y libasound2-dev

In [ ]:
!pip install pqdm

In [ ]:
import os,glob,re,math
import pandas as pd
import numpy as np
import librosa
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from pqdm.processes import pqdm
import cv2
import librosa
#from itertools import islice
import matplotlib
import tensorflow as tf
from scipy.signal import freqz
import torch
import warnings
warnings.filterwarnings('ignore')

In [ ]:
print(os.getcwd())
os.chdir('./pycochleagram')
print(os.getcwd())
!python setup.py install

In [ ]:
os.chdir('/kaggle/working')

In [ ]:
from distutils.dir_util import copy_tree
copy_tree("../input/cochleagramfile", "../working/")

In [ ]:
from pydub import AudioSegment

import cochleagram as cgram
import demo
import erbfilter as erb

from librosa.display import waveplot
from scipy.signal import butter, lfilter
from skimage.restoration import denoise_wavelet 
from scipy import signal

In [ ]:
ROOT_DIR = '../input/birdsong-recognition/'
TRAIN_AUDIO = f'{ROOT_DIR}/train_audio'

In [ ]:
CLASS = os.listdir('../input/birdsong-recognition/train_audio')

train_audio = glob.glob('../input/birdsong-recognition/train_audio/*/*.mp3')
train_audio_1 =glob.glob('../input/xeno-canto-bird-recordings-extended-a-m/A-M/*/*.mp3')
train_audio_2 = glob.glob('../input/xeno-canto-bird-recordings-extended-n-z/N-Z/*/*.mp3')

df = pd.read_csv('../input/birdsong-recognition/train.csv')

df = df[['ebird_code', 'filename', 'duration','author','country','rating']]
df1 = pd.read_csv('../input/xeno-canto-bird-recordings-extended-a-m/train_extended.csv')[['ebird_code', 'filename', 'duration','author','country','rating']]
df2 = pd.read_csv('../input/xeno-canto-bird-recordings-extended-n-z/train_extended.csv')[['ebird_code', 'filename', 'duration','author','country','rating']]

frames_temp = pd.concat([df1,df2])

dftemp = pd.read_csv('../input/freefield1010-cochlea/data/train_ff1010.csv')

list1 = ['Cornell Bird identification',len(set(df['ebird_code'])),len(df['ebird_code'])]
list2 = ['Xeno-canto',len(set(frames_temp['ebird_code'])),len(frames_temp['ebird_code'])]
list3 = ['freefeild1010','no call',len(dftemp['filename'])]
list4 = ['Cornell Bird identification + Xeno-canto + freefeild1010',str(len(set(frames_temp['ebird_code']))) + ' Species' +' and nocall',len(df['ebird_code'])+len(frames_temp['ebird_code'])+len(dftemp['filename'])]

dftable = pd.DataFrame([list1,list2,list3,list4],columns = ['Dataset','Number of classes/ bird species','Number of recordings'])

dftable.to_csv('dftable.csv')

frames = pd.concat([df,df1,df2])

path = train_audio + train_audio_1 + train_audio_2
path.remove('../input/birdsong-recognition/train_audio/lotduc/XC195038.mp3')
# path.remove('../input/xeno-canto-bird-recordings-extended-a-m/A-M/houspa/XC555482.mp3')

In [ ]:
class cochleagram:
    IMAGE_HEIGHT = 256
    IMAGE_WIDTH = 576
    SR=32000
    hi_lim = SR//2
    low_lim = 10
    n_filters = int(np.floor(erb.freq2erb(hi_lim) - erb.freq2erb(low_lim)) - 1)
    nonlinearity = 'db'
    ret_mode='envs',
    sample_factor = 2   # 87.5% overlap 2-75% verlap

# test

In [ ]:
signal, sr = librosa.load('../input/birdsong-recognition/train_audio/aldfly/XC134874.mp3',sr = None,duration = 10)

human_co = cgram.human_cochleagram(signal,sr =cochleagram.SR,
                             n = cochleagram.n_filters,
                             low_lim = cochleagram.low_lim, hi_lim =cochleagram.hi_lim,
                             sample_factor = cochleagram.sample_factor,
                             nonlinearity=cochleagram.nonlinearity)

import cv2
res = cv2.resize(np.flipud(human_co), dsize=(576,256), interpolation=cv2.INTER_CUBIC)

plt.imshow(res, aspect='auto')

In [ ]:
config_patch = {'use_inv_stem' : True,
          'use_patch' : [True,16],}

In [ ]:
def inv_stem(x):
    print(x.shape)
    x1 = x.transpose(0, 1).view(24, 24, 16, 16)
    y = torch.zeros(384, 384, dtype=x.dtype)
    for i in range(24):
        for j in range(24):
            y[i*16:(i+1)*16, j*16:(j+1)*16] = x1[i, j]
    return y

In [ ]:
def image_patch(mel_spect):
    if config_patch.get('use_inv_stem'):
#         spect = torch.from_numpy(np.flip(mel_spect).copy())
        spect = torch.from_numpy(mel_spect.copy())
        spect = inv_stem(spect)
#         print(spect.shape)
    else:
        if config_patch.get('use_patch')[0]:
            patch_size = config_patch.get('use_patch')[1]
            spect = np.zeros((384, 384), dtype=np.float32)
        for i in range(0, 192, patch_size):
            spect[2 * i : 2 * i + patch_size, :] = mel_spect[i : i + patch_size, : 384]
            spect[2 * i + patch_size : 2 * i + 2*patch_size, :] = mel_spect[i : i + patch_size, 384 : ]
            mel_spect = spect
    return spect.cpu().detach().numpy()

In [ ]:
plt.imshow(image_patch(res), aspect='auto')

# test end

In [ ]:
def cochlea(file_path):
    signal, sr = librosa.load(file_path,sr = None,duration = 10)
    human_co = cgram.human_cochleagram(signal,sr =cochleagram.SR,
                             n = cochleagram.n_filters,
                             low_lim = cochleagram.low_lim, hi_lim =cochleagram.hi_lim,
                             sample_factor = cochleagram.sample_factor,
                             nonlinearity=cochleagram.nonlinearity)
    return human_co

In [ ]:
config_patch = {'use_inv_stem' : True,
          'use_patch' : [True,16],}

In [ ]:
def inv_stem(x):
    print(x.shape)
    x1 = x.transpose(0, 1).view(24, 24, 16, 16)
    y = torch.zeros(384, 384, dtype=x.dtype)
    for i in range(24):
        for j in range(24):
            y[i*16:(i+1)*16, j*16:(j+1)*16] = x1[i, j]
    return y

In [ ]:
def image_patch(mel_spect):
    if config_patch.get('use_inv_stem'):
#         spect = torch.from_numpy(np.flip(mel_spect).copy())
        spect = torch.from_numpy(mel_spect.copy())
        spect = inv_stem(spect)
#         print(spect.shape)
    else:
        if config_patch.get('use_patch')[0]:
            patch_size = config_patch.get('use_patch')[1]
            spect = np.zeros((384, 384), dtype=np.float32)
        for i in range(0, 192, patch_size):
            spect[2 * i : 2 * i + patch_size, :] = mel_spect[i : i + patch_size, : 384]
            spect[2 * i + patch_size : 2 * i + 2*patch_size, :] = mel_spect[i : i + patch_size, 384 : ]
            mel_spect = spect
    return spect.cpu().detach().numpy()

In [ ]:
list_allgroup = []
seq = path
subStr_list = [ f.name for f in os.scandir('../input/birdsong-recognition/train_audio') if f.is_dir() ]

for name in subStr_list:
    list_temp = []
    for text in path:
        if name in text:
            list_temp.append(text)
    list_allgroup.append(list_temp)

In [ ]:
list_allgroup = sorted(list_allgroup)

In [ ]:
len(list_allgroup)

In [ ]:
list_chucks = []

four_split = np.array_split(list_allgroup,12 )
for array in four_split:
    list_chucks.append((list(array)))

In [ ]:
list_allgroup = list_chucks[0]

In [ ]:
output_root_dir = "cochleagram"
os.makedirs(os.path.join(output_root_dir), exist_ok=True)

In [ ]:
import shutil
  
src = '../input/freefield1010-cochlea/data/nocall'
trg = './cochleagram/nocall/'

os.makedirs(trg, exist_ok=True)

files=os.listdir(src)
 
# iterating over all the files in
# the source directory
for fname in files:
    shutil.copy2(os.path.join(src,fname), trg)

In [ ]:
def long_substr(data):
    substrs = lambda x: {x[i:i+j] for i in range(len(x)) for j in range(len(x) - i + 1)}
    s = substrs(data[0])
    for val in data[1:]:
        s.intersection_update(substrs(val))
    return max(s, key=len).split('/')[1].strip()

In [ ]:
list_allgroup[0][0]

In [ ]:
len(list_allgroup)

In [ ]:
import pathlib

In [ ]:
for sub_list in list_allgroup:
# def final_cochlea(sub_list):
    x_bird = pathlib.PurePath(sub_list[0])
    bird_name = x_bird.parent.name
    print(bird_name)
    os.makedirs(os.path.join(output_root_dir,bird_name), exist_ok=True)
    for path_file in sub_list:
        fileName = os.path.basename(path_file)[:-4]
        human_co = cochlea(path_file)
        res = cv2.resize(np.flipud(human_co), dsize=(576,256), interpolation=cv2.INTER_CUBIC)
        img = image_patch(res)
        matplotlib.image.imsave(os.path.join(os.path.join(output_root_dir,bird_name,"{}.jpg".format(fileName))), img*255)
#         cv2.imwrite(os.path.join(os.path.join(output_root_dir,bird_name,"{}.jpg".format(fileName))),img*255)

In [ ]:
# res = pqdm(list_allgroup, final_cochlea, n_jobs=8)

In [ ]:
os.listdir('./cochleagram')

In [ ]:
# coch = demo.demo_human_cochleagram_helper(signal,sr,50, nonlinearity=None)
# coch_log = demo.demo_human_cochleagram_helper(signal,sr,50,nonlinearity='db')
# coch_pow = demo.demo_human_cochleagram_helper(signal,sr,50,nonlinearity='power' )

In [ ]:
# import utils
# plt.figure(figsize=(10, 4))
# plt.title('Cochleagram with no nonlinearity')
# plt.ylabel('Filter #')
# plt.xlabel('time')
# utils.cochshow(np.flipud(coch), interact=False, cmap = 'viridis')
# plt.gca().invert_yaxis()
# plt.savefig('Cochleagram with no nonlinearity.png', bbox_inches='tight',transparent = True)


# plt.figure(figsize=(10, 4))
# plt.title('Cochleagram with nonlinearity: decibel')
# plt.ylabel('Filter #')
# plt.xlabel('time')
# utils.cochshow(np.flipud(coch_log), interact=False, cmap = 'viridis')
# plt.gca().invert_yaxis()
# plt.savefig('Cochleagram with no nonlinearity_log.png', bbox_inches='tight',transparent = True)


# plt.figure(figsize=(10, 4))
# plt.title('Cochleagram with nonlinearity: power')
# plt.ylabel('Filter #')
# plt.xlabel('time')
# utils.cochshow(np.flipud(coch_pow), interact=False, cmap = 'viridis')
# plt.gca().invert_yaxis()
# plt.savefig('Cochleagram with no nonlinearity_pow.png', bbox_inches='tight',transparent = True)

# Done